In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import copy
%matplotlib inline

In [3]:
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Merkle Sokrati Assignment/Data Clean 1.xlsx')

In [4]:
df.head(2)

,Date,product,phase,campaign_platform,campaign_type,communication_medium,subchannel,audience_type,creative_type,creative_name,device,age,spends,impressions,clicks,link_clicks
0,2019-10-16,Product 1,Performance,Google Ads,Search,Search Keywords,Brand,'-,'-,'-,Desktop,18-24,0.0,0,0,0
1,2019-10-16,Product 1,Performance,Google Ads,Search,Search Keywords,Brand,'-,'-,'-,Desktop,25-34,0.0,5,0,0


In [ ]:
print(df.info())

In [ ]:
# df.describe

In [6]:
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import abline_plot

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
cat_df = df.select_dtypes(include=['object']).copy()

In [8]:
cat_df.head()

,product,phase,campaign_platform,campaign_type,communication_medium,subchannel,audience_type,creative_type,creative_name,device,age
0,Product 1,Performance,Google Ads,Search,Search Keywords,Brand,'-,'-,'-,Desktop,18-24
1,Product 1,Performance,Google Ads,Search,Search Keywords,Brand,'-,'-,'-,Desktop,25-34
2,Product 1,Performance,Google Ads,Search,Search Keywords,Brand,'-,'-,'-,Desktop,35-44
3,Product 1,Performance,Google Ads,Search,Search Keywords,Brand,'-,'-,'-,Desktop,Undetermined
4,Product 1,Performance,Google Ads,Search,Search Keywords,Brand,'-,'-,'-,Desktop,18-24


In [9]:
print(cat_df.isnull().values.sum())

0


In [10]:
cat_df_onehot = cat_df.copy()
cat_df_onehot = pd.get_dummies(cat_df_onehot, columns=['campaign_platform'], prefix = ['campaign_platform'])
cat_df_onehot = pd.get_dummies(cat_df_onehot, columns=['campaign_type'], prefix = ['campaign_type'])
cat_df_onehot = pd.get_dummies(cat_df_onehot, columns=['communication_medium'], prefix = ['communication_medium'])
cat_df_onehot = pd.get_dummies(cat_df_onehot, columns=['subchannel'], prefix = ['subchannel'])
cat_df_onehot = pd.get_dummies(cat_df_onehot, columns=['audience_type'], prefix = ['audience_type'])
cat_df_onehot = pd.get_dummies(cat_df_onehot, columns=['creative_type'], prefix = ['creative_type'])
cat_df_onehot = pd.get_dummies(cat_df_onehot, columns=['creative_name'], prefix = ['creative_name'])
cat_df_onehot = pd.get_dummies(cat_df_onehot, columns=['device'], prefix = ['device'])
cat_df_onehot = pd.get_dummies(cat_df_onehot, columns=['age'], prefix = ['age'])

print(cat_df_onehot.head())

     product        phase  ...  age_65 or more  age_Undetermined
0  Product 1  Performance  ...               0                 0
1  Product 1  Performance  ...               0                 0
2  Product 1  Performance  ...               0                 0
3  Product 1  Performance  ...               0                 1
4  Product 1  Performance  ...               0                 0

[5 rows x 34 columns]


In [11]:
conti_df = df.select_dtypes(include=['float64', 'int64']).copy()
conti_df.head()

,spends,impressions,clicks,link_clicks
0,0.00,0,0,0
1,0.00,5,0,0
2,0.00,1,0,0
3,14.63,5,3,0
4,53.31,10,2,0


In [12]:
result_df = pd.concat([cat_df_onehot, conti_df], axis=1)

print(result_df.head())

     product        phase  ...  clicks  link_clicks
0  Product 1  Performance  ...       0            0
1  Product 1  Performance  ...       0            0
2  Product 1  Performance  ...       0            0
3  Product 1  Performance  ...       3            0
4  Product 1  Performance  ...       2            0

[5 rows x 38 columns]


In [13]:
# Dropping irrelevant columns
result_df = result_df.drop(['product', 'phase'], axis=1)
result_df.head(5)

,campaign_platform_Facebook Ads,campaign_platform_Google Ads,campaign_type_Conversions,campaign_type_Search,communication_medium_Creative,communication_medium_Search Keywords,subchannel_Brand,subchannel_Competitor,subchannel_Facebook Ads,subchannel_Generic,audience_type_'-,audience_type_Audience 1,audience_type_Audience 2,audience_type_Audience 3,creative_type_'-,creative_type_Carousal,creative_type_Image,creative_name_'-,creative_name_Carousal,creative_name_Click,creative_name_Girl,device_Desktop,device_Mobile,device_Tablet,device_device,age_18-24,age_25-34,age_35-44,age_45-54,age_55-64,age_65 or more,age_Undetermined,spends,impressions,clicks,link_clicks
0,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0.00,0,0,0
1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0.00,5,0,0
2,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0.00,1,0,0
3,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,14.63,5,3,0
4,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,53.31,10,2,0


In [14]:
print(result_df.shape)

(16834, 36)


In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [16]:
X = result_df.iloc[:, :-1].values
Y = result_df.iloc[:, -1].values

print(X[-1])

[ 0.   1.   0.   1.   0.   1.   1.   0.   0.   0.   1.   0.   0.   0.
  1.   0.   0.   1.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.
  0.   0.   0.   1.   1.4 14.   1. ]


In [17]:
# Fit and make the predictions by the model
model = sm.OLS(Y, X).fit()
predictions = model.predict(X)

In [18]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                 2.088e+04
Date:                Sat, 22 Aug 2020   Prob (F-statistic):               0.00
Time:                        15:15:21   Log-Likelihood:                -46353.
No. Observations:               16834   AIC:                         9.274e+04
Df Residuals:                   16815   BIC:                         9.289e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.3213      0.019     16.837      0.000       0.284       0.359
x2            -0.0592      0.007     -9.062      0.000      -0.072      -0.046
x3             0.3213      0.019     16.837      0.000       0.284       0.359
x4            -0.0592      0.007     -9.062      0.000      -0.072      -0.046
x5             0.3213      0.019     16.837      0.000       0.284       0.359
x6            -0.0592      0.007     -9.062      0.000      -0.072      -0.046
x7            -0.5573      0.042    -13.194      0.000      -0.640      -0.474
x8             0.5480      0.055      9.885      0.000       0.439       0.657
x9             0.3213      0.019     16.837      0.000       0.284       0.359
x10           -0.0500      0.048     -1.040      0.299      -0.144       0.044
x11           -0.0592      0.007     -9.062      0.000      -0.072      -0.046
x12            1.1739      0.140      8.399      0.000       0.900       1.448
x13            0.1719      0.139      1.240      0.215      -0.100       0.444
x14           -1.0245      0.204     -5.027      0.000      -1.424      -0.625
x15           -0.0592      0.007     -9.062      0.000      -0.072      -0.046
x16           -0.1297      0.058     -2.243      0.025      -0.243      -0.016
x17            0.4510      0.056      8.028      0.000       0.341       0.561
x18           -0.0592      0.007     -9.062      0.000      -0.072      -0.046
x19           -0.1297      0.058     -2.243      0.025      -0.243      -0.016
x20            2.4333      0.121     20.136      0.000       2.196       2.670
x21           -1.9823      0.123    -16.161      0.000      -2.223      -1.742
x22            0.2320      0.048      4.839      0.000       0.138       0.326
x23           -1.0453      0.049    -21.325      0.000      -1.141      -0.949
x24            0.7541      0.074     10.206      0.000       0.609       0.899
x25            0.3213      0.019     16.837      0.000       0.284       0.359
x26           -0.2978      0.073     -4.090      0.000      -0.441      -0.155
x27           -0.1458      0.069     -2.124      0.034      -0.280      -0.011
x28           -0.0129      0.067     -0.192      0.848      -0.145       0.119
x29            0.1768      0.072      2.458      0.014       0.036       0.318
x30            0.2534      0.084      3.008      0.003       0.088       0.418
x31            0.8845      0.089      9.946      0.000       0.710       1.059
x32           -0.5961      0.071     -8.356      0.000      -0.736      -0.456
x33           -0.0072      0.000    -61.262      0.000      -0.007      -0.007
x34            0.0048   3.23e-05    148.046      0.000       0.005       0.005
x35            0.1965      0.002     90.725      0.000       0.192       0.201
==============================================================================
Om

In [19]:
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()
predictions = model.predict(X)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                 2.088e+04
Date:                Sat, 22 Aug 2020   Prob (F-statistic):               0.00
Time:                        15:15:23   Log-Likelihood:                -46353.
No. Observations:               16834   AIC:                         9.274e+04
Df Residuals:                   16815   BIC:                         9.289e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2019      0.015     13.261      0.000       0.172       0.232
x1             0.2901      0.017     17.187      0.000       0.257       0.323
x2            -0.0882      0.006    -13.984      0.000      -0.101      -0.076
x3             0.2901      0.017     17.187      0.000       0.257       0.323
x4            -0.0882      0.006    -13.984      0.000      -0.101      -0.076
x5             0.2901      0.017     17.187      0.000       0.257       0.323
x6            -0.0882      0.006    -13.984      0.000      -0.101      -0.076
x7            -0.5669      0.042    -13.405      0.000      -0.650      -0.484
x8             0.5384      0.055      9.723      0.000       0.430       0.647
x9             0.2901      0.017     17.187      0.000       0.257       0.323
x10           -0.0596      0.048     -1.240      0.215      -0.154       0.035
x11           -0.0882      0.006    -13.984      0.000      -0.101      -0.076
x12            1.1635      0.140      8.309      0.000       0.889       1.438
x13            0.1615      0.139      1.163      0.245      -0.111       0.434
x14           -1.0349      0.203     -5.088      0.000      -1.434      -0.636
x15           -0.0882      0.006    -13.984      0.000      -0.101      -0.076
x16           -0.1431      0.058     -2.484      0.013      -0.256      -0.030
x17            0.4332      0.056      7.723      0.000       0.323       0.543
x18           -0.0882      0.006    -13.984      0.000      -0.101      -0.076
x19           -0.1431      0.058     -2.484      0.013      -0.256      -0.030
x20            2.4244      0.121     20.063      0.000       2.188       2.661
x21           -1.9912      0.123    -16.235      0.000      -2.232      -1.751
x22            0.2224      0.048      4.629      0.000       0.128       0.317
x23           -1.0549      0.049    -21.482      0.000      -1.151      -0.959
x24            0.7444      0.074     10.093      0.000       0.600       0.889
x25            0.2901      0.017     17.187      0.000       0.257       0.323
x26           -0.3064      0.073     -4.212      0.000      -0.449      -0.164
x27           -0.1544      0.069     -2.248      0.025      -0.289      -0.020
x28           -0.0215      0.067     -0.319      0.750      -0.154       0.111
x29            0.1682      0.072      2.337      0.019       0.027       0.309
x30            0.2448      0.084      2.907      0.004       0.080       0.410
x31            0.8759      0.089      9.862      0.000       0.702       1.050
x32           -0.6047      0.071     -8.484      0.000      -0.744      -0.465
x33           -0.0072      0.000    -61.262      0.000      -0.007      -0.007
x34            0.0048   3.23e-05    148.046      0.000       0.005       0.005
x35            0.1965      0.002     90.725      0.000       0.192       0.201
==

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [24]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [25]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[-2.33  0.  ]
 [-0.09  0.  ]
 [ 8.61  0.  ]
 ...
 [-0.09  0.  ]
 [-1.04  0.  ]
 [-0.86  0.  ]]


In [26]:
regressor.score(X,Y)

0.9571219715709328

In [27]:
regressor.coef_

array([-1.10e-14,  1.92e-01, -1.92e-01,  1.92e-01, -1.92e-01,  1.92e-01,
       -1.92e-01, -6.03e-01,  4.96e-01,  1.92e-01, -8.52e-02, -1.92e-01,
        9.06e-01,  2.99e-01, -1.01e+00, -1.92e-01, -1.38e-01,  3.30e-01,
       -1.92e-01, -1.38e-01,  2.41e+00, -2.08e+00,  1.88e-01, -1.06e+00,
        6.77e-01,  1.92e-01, -3.06e-01, -8.87e-02, -6.86e-02,  1.12e-01,
        1.78e-01,  8.13e-01, -6.39e-01, -7.21e-03,  4.86e-03,  1.91e-01])